In [8]:
import numpy as np
import pandas as pd
from datetime import datetime

In [26]:
df_st = pd.read_csv('data/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv',encoding='euc-kr')
df_st['지하철역'] = df_st['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df_st.index:
    if df_st.loc[i, '지하철역'][-1] == '역':
        df_st.loc[i, '지하철역'] = df_st.loc[i, '지하철역'][:-1]
now = datetime.now()
f_date = now.strftime('%Y%m%d')
df_st.to_csv(f'data/{f_date}_info.csv',index=False,encoding='utf-8')

In [27]:
df = pd.read_csv(f'data/{f_date}_info.csv')
df.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202304,1호선,동대문,749,13,10603,2210,9478,6645,14847,...,8762,524,2480,1,186,0,0,0,0,20230503
1,202304,1호선,동묘앞,204,3,2738,993,3544,4933,7427,...,3998,89,1634,2,295,0,0,0,0,20230503
2,202304,1호선,서울,673,28,7682,8982,13316,56520,42256,...,13801,2424,2423,0,179,0,0,0,0,20230503
3,202304,1호선,시청,135,5,2141,4360,3966,24036,7878,...,4147,345,923,0,14,0,0,0,0,20230503
4,202304,1호선,신설동,409,5,8266,2287,9839,8406,21340,...,9184,395,1967,0,0,1,1,0,0,20230503


In [ ]:
############### 수정중

In [25]:
# 호선명 리스트로 추출해서 사용 
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    # frame = df[df['호선명']==line].copy()
    frame['새벽 승차인원'] = frame.loc[:,['04시-05시 승차인원','05시-06시 승차인원']].sum(axis=1)
    frame['새벽 하차인원'] = frame.loc[:,['04시-05시 하차인원','05시-06시 하차인원']].sum(axis=1)

    frame['출근시간 승차인원'] = frame.loc[:,['06시-07시 승차인원','07시-08시 승차인원','08시-09시 승차인원']].sum(axis=1)
    frame['09-16시 승차인원'] = frame.loc[:,['09시-10시 승차인원','10시-11시 승차인원','11시-12시 승차인원',
                                            '12시-13시 승차인원','13시-14시 승차인원','14시-15시 승차인원','16시-17시 승차인원']].sum(axis=1)
    frame['퇴근시간 승차인원'] = frame.loc[:,['17시-18시 승차인원','18시-19시 승차인원','19시-20시 승차인원']].sum(axis=1)
    frame['야간 승차인원'] = frame.loc[:,['20시-21시 승차인원','21시-22시 승차인원','22시-23시 승차인원',
                                        '23시-24시 승차인원','00시-01시 승차인원','01시-02시 승차인원']].sum(axis=1)
    frame['출근시간 하차인원'] = frame.loc[:,['06시-07시 하차인원','07시-08시 하차인원','08시-09시 하차인원']].sum(axis=1)
    frame['퇴근시간 하차인원'] = frame.loc[:,['17시-18시 하차인원','18시-19시 하차인원','19시-20시 하차인원']].sum(axis=1)
    frame['09-16시 하차인원'] = frame.loc[:,['09시-10시 하차인원','10시-11시 하차인원','11시-12시 하차인원',
                                            '12시-13시 하차인원','13시-14시 하차인원','14시-15시 하차인원','16시-17시 하차인원']].sum(axis=1)
    frame['야간 하차인원'] = frame.loc[:,['20시-21시 하차인원','21시-22시 하차인원','22시-23시 하차인원',
                                        '23시-24시 하차인원','00시-01시 하차인원','01시-02시 하차인원']].sum(axis=1)
    frame['총 승차인원'] = frame.loc[:,['새벽 승차인원','출근시간 승차인원','09-16시 승차인원','야간 승차인원']].sum(axis=1)
    frame['총 하차인원'] = frame.loc[:,['새벽 하차인원','출근시간 하차인원','09-16시 하차인원','야간 하차인원']].sum(axis=1)
    frame = frame[['사용월', '호선명', '지하철역', '출근시간 승차인원', '출근시간 하차인원', 
                    '09-16시 승차인원', '09-16시 하차인원', '퇴근시간 승차인원', '퇴근시간 하차인원',
                    '새벽 승차인원','새벽 하차인원','야간 승차인원', '야간 하차인원','총 승차인원','총 하차인원']]
    
    frame.loc[(frame['호선명'] == '2호선') & (frame['지하철역'] == '신천'), '지하철역'] = '잠실새내'

    frame.to_csv(f'data/main/temp_files/{line}.csv',index=False,encoding='utf-8')